In [1]:
# Import libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import Sequence

In [2]:
# Function to load images 
def load_image_paths(data_dir):
    image_paths = []
    labels = []
    class_names = sorted(os.listdir(data_dir))
    
    for class_idx, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_dir):
            continue
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            if os.path.isfile(img_path):  # Ensure it's a file
                image_paths.append(img_path)
                labels.append(class_name)
    
    return image_paths, labels, class_names

In [3]:
# Load and preprocess data
data_dir = r'C:\Users\hp\Desktop\digital skills\datasets'  # Path to your dataset folder
image_paths, labels, class_names = load_image_paths(data_dir)
print(f"Loaded {len(image_paths)} images across {len(class_names)} classes: {class_names}")

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
encoded_labels = to_categorical(encoded_labels)

# Fixed DataGenerator class
class DataGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size, shuffle=True):
        self.image_paths = np.array(image_paths)
        self.labels = np.array(labels)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))
    
    def __getitem__(self, index):
        start_idx = index * self.batch_size
        end_idx = min((index + 1) * self.batch_size, len(self.image_paths))
        batch_paths = self.image_paths[start_idx:end_idx]
        batch_labels = self.labels[start_idx:end_idx]
        
        images = []
        valid_labels = []
        
        for path, label in zip(batch_paths, batch_labels):
            img = cv2.imread(path)
            if img is None:
                print(f"Warning: Could not load image {path}")
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))  # Changed to 224x224
            images.append(img.astype(np.float32) / 255.0)
            valid_labels.append(label)
        
        return np.array(images), np.array(valid_labels)
    
    def on_epoch_end(self):
        if self.shuffle:
            indices = np.random.permutation(len(self.image_paths))
            self.image_paths = self.image_paths[indices]
            self.labels = self.labels[indices]

# Create generators - split encoded_labels instead of string labels
train_paths, val_paths, train_encoded, val_encoded = train_test_split(
    image_paths, encoded_labels, test_size=0.2, random_state=42
)

train_generator = DataGenerator(train_paths, train_encoded, batch_size=16)
val_generator = DataGenerator(val_paths, val_encoded, batch_size=16, shuffle=False)

Loaded 1856 images across 2 classes: ['Tajin', 'kaftan']


In [4]:
# Build CNN model (adjusted to 224x224 input)
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(class_names), activation='softmax')  # Output layer matches class count
])

# Compile model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 32)        │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 43264)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │       2,768,960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,792,674 (10.65 MB)

 Trainable params: 2,792,674 (10.65 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=100,  # Reduced from 100 to speed up initial training
    verbose=1
)

C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
20/93 ━━━━━━━━━━━━━━━━━━━━ 50s 692ms/step - accuracy: 0.6188 - loss: 0.9702Warning: Could not load image C:\Users\hp\Desktop\digital skills\datasets\kaftan\ezgif-4-b0c9cca3e6.gif
53/93 ━━━━━━━━━━━━━━━━━━━━ 26s 670ms/step - accuracy: 0.6759 - loss: 0.7760Warning: Could not load image C:\Users\hp\Desktop\digital skills\datasets\Tajin\couscous-aux-légumes-et-viande-tajine.jpg
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 707ms/step - accuracy: 0.7170 - loss: 0.6699Warning: Could not load image C:\Users\hp\Desktop\digital skills\datasets\kaftan\mop.gif
93/93 ━━━━━━━━━━━━━━━━━━━━ 93s 957ms/step - accuracy: 0.7178 - loss: 0.6678 - val_accuracy: 0.8787 - val_loss: 0.2905
Epoch 2/100
23/93 ━━━━━━━━━━━━━━━━━━━━ 50s 721ms/step - accuracy: 0.8690 - loss: 0.3091Warning: Could not load image C:\Users\hp\Desktop\digital skills\datasets\Tajin\couscous-aux-légumes-et-viande-tajine.jpg
68/93 ━━━━━━━━━━━━━━━━━━━━ 19s 777ms/step - accuracy: 0.8882 - loss: 0.2671Warning: Could not load image C:\Users\hp\Deskt

In [ ]:
# Save model and label encoder
model.save('image_name_predictor.h5')
np.save('label_encoder_classes.npy', label_encoder.classes_)

# Evaluate model
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_accuracy:.4f}")

In [ ]:
# Plot training and validation accuracy/loss
plt.figure(figsize=(12, 4))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()